<a href="https://colab.research.google.com/github/BenUCL/Reef-acoustics-and-AI/blob/main/Code/Random_forests_pretrained_CNN_Indo_site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Random forest classifiers**

This script splits the data into the same train/val/test sets as used for the pretrained CNN and trained CNN. It then trains on the training data and generates the accuracy for the validation data 50 times. The best performing model of the 50 repeats is used to inference on the test data and this accuracy is reported. This proces is repeatd 100 times to generate an accuracyfor all train/val/test split combinations. A confusion matrix of all these is then generated.

In [1]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#imports
import sklearn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd
from sklearn.model_selection import train_test_split
import statistics
from sklearn.metrics import confusion_matrix

In [2]:
# imports, probably some redundant packages
import sklearn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd
from sklearn.model_selection import train_test_split
import statistics

from sklearn.metrics import confusion_matrix

import collections
from sklearn.metrics import ConfusionMatrixDisplay

import seaborn as sn
import matplotlib.pyplot as plt

In [3]:
# Upload the csv of features and copy the path
path = r'/content/drive/MyDrive/Reef soundscapes with AI/Results/full_dataset_features/pretrained_CNN_indo.csv'
num_classes = 4
labels = ['HBa','HBo','DBo','DSa']

In [4]:
data = pd.read_csv (path) #load dataframe
temp_df = data.reset_index() #put index in order
temp_df = temp_df.iloc[: , 2:] #remove unnecessary index
temp_df = temp_df.T #transpose to match indices format
temp_df = temp_df.reset_index() #re-add the index
df = temp_df.rename(columns={"index": "minute"})

df

,minute,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,BoF2.0930D.1678278701.180828.NT0930.wav,0.002933,0.177240,0.388430,0.129290,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.041895,0.000000,0.000000,0.000000,0.433386,0.320776,0.000000,0.000000,0.0
1,SaF4.0902D.671907872.180830.NT0924.wav,0.000917,0.073721,0.515714,0.111717,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.166053,0.000093,0.000208,0.048393,0.167975,0.032886,0.000000,0.000000,0.0
2,SaF3.1355D.805322778.180829.NT1356.wav,0.023495,0.104395,0.360114,0.143027,0.000000,0.0,0.0,0.000855,0.0,...,0.0,0.104303,0.036998,0.001894,0.006081,0.209761,0.121807,0.000000,0.000000,0.0
3,SaN11.0940D.1678278701.180906.NT0954.wav,0.000865,0.126758,0.437884,0.209243,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.117832,0.001151,0.011284,0.000152,0.328505,0.123348,0.000000,0.007058,0.0
4,BoF2.0930D.1678278701.180828.NT0950.wav,0.019745,0.176513,0.399040,0.040165,0.000027,0.0,0.0,0.006952,0.0,...,0.0,0.020741,0.020154,0.003591,0.003634,0.350962,0.285468,0.002281,0.000323,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331,BoF3.1205H.1678278701.180829.NT1231.wav,0.010470,0.125127,0.444852,0.083505,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.023687,0.000918,0.000000,0.000753,0.493698,0.257412,0.000423,0.000000,0.0
3332,BoF5.0940D.1677983769.180831.NT1007.wav,0.006456,0.129605,0.418061,0.163294,0.000000,0.0,0.0,0.001106,0.0,...,0.0,0.085375,0.000477,0.000000,0.000180,0.326792,0.223547,0.000000,0.000000,0.0
3333,BoF3.1205H.1678278701.180829.NT1205.wav,0.001636,0.122509,0.458245,0.078943,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.022889,0.000000,0.000000,0.000000,0.514317,0.261960,0.000536,0.000000,0.0
3334,BoF5.0940H.671907872.180831.NT1035.wav,0.008864,0.049046,0.569985,0.045106,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.140025,0.000000,0.000000,0.095348,0.208568,0.032443,0.000000,0.000000,0.0


In [5]:
'''This code block find the unique identifiers of each deployment (i.e what
hour of the day at what site) and splits these ID's into training, val and
test sets. These are used in the next codeblock to select the actual 
recordings using these ID's which are present within each minute from the
same deployment'''


#This function takes the parts of a filename that make it unique
 #This uses Tims naming convention, specific to the 2018 Indonesia data
def get_identifier(filename):
    #find part of the name that corresponds to the deployment
    t0 = filename.split(".")[0]
    t1 = filename.split(".")[1][0:5]
    t = t0+'.'+t1
    return t
 
#Function to get unique values within an array
def unique(list1):
    x = np.array(list1)
    return np.unique(x)

def get_class(filename):
    #find part of the name that corresponds to the deployment
     #adapted the get_identifier function above to only get class (e.g healthy)
    deployment_ID = filename.split(".")[1][4:5]
    return deployment_ID

def train_test_val_split(df):
  IDs = []
  for index, row in df.iterrows():
      filename = (df['minute'][index])
      IDs.append(get_identifier(filename))
  all_IDs = unique(IDs)

  #Use the above function to get a list of unique deployment ID's (approx 30 for healthy, and again for degraded)
  unshuffled_unique_deployments = unique(all_IDs) #so for the real data this will give a big long order list

  #ensure the same random shuffle is made each time for a CV, so the order is conserved across the 3 methods
  np.random.seed(repeat) 
  #print(repeat)

  #shuffle this list
  shuffled_unique_deployments = np.random.permutation(unshuffled_unique_deployments)
  #np.random.seed() #now lift the seed so that randomisation can be used again in the rest of the script


  #Create df of all deployments and their class
  d = {'Deployment': shuffled_unique_deployments }
  d1 = pd.DataFrame(data=d)

  df_withclasses = d1

  # add a column to the DF that contains class
  new_list = []
  for i in range(len(df_withclasses)):
      new_list.append(get_class(df_withclasses['Deployment'][i]))

  df_withclasses.insert(1, 'Class', new_list)

  # Use the new df to split the deployment ID's into train/val/test sets
  # pick 15% (8 deployments) of data as test data
  validation, the_rest = sklearn.model_selection.train_test_split(df_withclasses, test_size=0.85, stratify=df_withclasses['Class']) 
  # pick 0.15*0.85% (8 deployments) of the_rest to be the val data
  train, test = sklearn.model_selection.train_test_split(the_rest, test_size=0.15, stratify=the_rest['Class']) 

  # convert these to numpy arrays
  train_deployments = np.array(train['Deployment'])
  val_deployments = np.array(validation['Deployment'])
  test_deployments = np.array(test['Deployment'])   
  np.random.seed() #now lift the seed so that randomisation can be used again in the rest of the script

  return train_deployments, val_deployments, test_deployments



In [6]:
def put_files_in_splits(train_deployments, val_deployments, test_deployments):
  '''This block uses the ID's of the train, val and test sets generated above
  to select the actual recordings. This generates:
  train_files
  val_files
  test_files
  Which are the arrays of recordings corresponding to each of these '''


  #now select all files in dir that have these ID's in their name
  train_files = []
  val_files = []
  test_files = []


  #for f in all_files: #I set this above 

  for index, row in df.iterrows():
    filename = (df['minute'][index]) 
    
    #print(f)
    namePt1 = filename.split(".")[0]
    namePt2 = filename.split(".")[1]
    ID = namePt1 + '.' + namePt2
    #print(ID)
    if ID in train_deployments:
      train_files.append(filename)
    if ID in val_deployments:
      val_files.append(filename)
    if ID in test_deployments:
      test_files.append(filename)

  return train_files, val_files, test_files

In [7]:
def get_class(filename):
    #find part of the name that corresponds to the deployment
     #adapted the get_identifier function above to only get class (e.g healthy)
    pt1 = filename[0:2]
    pt2 = filename.split(".")[1][4:5]
    site = pt2 + pt1
    return site

df_withclasses = df

# add a column to the DF that contains class
new_list = []
for i in range(len(df_withclasses)):
    new_list.append(get_class(df_withclasses['minute'][i]))

df_withclasses.insert(1, 'Class', new_list)
df_withclasses.head(20)

,minute,Class,0,1,2,3,4,5,6,7,...,118,119,120,121,122,123,124,125,126,127
0,BoF2.0930D.1678278701.180828.NT0930.wav,DBo,0.002933,0.177240,0.388430,0.129290,0.000000,0.0,0.0,0.000000,...,0.0,0.041895,0.000000,0.000000,0.000000,0.433386,0.320776,0.000000,0.000000,0.0
1,SaF4.0902D.671907872.180830.NT0924.wav,DSa,0.000917,0.073721,0.515714,0.111717,0.000000,0.0,0.0,0.000000,...,0.0,0.166053,0.000093,0.000208,0.048393,0.167975,0.032886,0.000000,0.000000,0.0
2,SaF3.1355D.805322778.180829.NT1356.wav,DSa,0.023495,0.104395,0.360114,0.143027,0.000000,0.0,0.0,0.000855,...,0.0,0.104303,0.036998,0.001894,0.006081,0.209761,0.121807,0.000000,0.000000,0.0
3,SaN11.0940D.1678278701.180906.NT0954.wav,DSa,0.000865,0.126758,0.437884,0.209243,0.000000,0.0,0.0,0.000000,...,0.0,0.117832,0.001151,0.011284,0.000152,0.328505,0.123348,0.000000,0.007058,0.0
4,BoF2.0930D.1678278701.180828.NT0950.wav,DBo,0.019745,0.176513,0.399040,0.040165,0.000027,0.0,0.0,0.006952,...,0.0,0.020741,0.020154,0.003591,0.003634,0.350962,0.285468,0.002281,0.000323,0.0
5,SaF3.2333D.805322778.180829.NT0020.wav,DSa,0.000094,0.124583,0.458020,0.177560,0.000000,0.0,0.0,0.000000,...,0.0,0.089366,0.000000,0.005892,0.005609,0.182370,0.108468,0.000000,0.000769,0.0
6,BaF3.0915H.1678278701.180829.NT0920.wav,HBa,0.017756,0.059928,0.645120,0.034384,0.000000,0.0,0.0,0.000000,...,0.0,0.129295,0.004092,0.000000,0.080960,0.315314,0.086141,0.000000,0.000000,0.0
7,BoF5.0940D.1677983769.180831.NT1028.wav,DBo,0.010602,0.121170,0.416814,0.120820,0.000000,0.0,0.0,0.000711,...,0.0,0.073866,0.001419,0.000000,0.002679,0.335018,0.203831,0.000000,0.000000,0.0
8,BaN11.1315H.805322778.180906.NT1339.wav,HBa,0.016682,0.084986,0.404000,0.098454,0.000000,0.0,0.0,0.000000,...,0.0,0.048633,0.000031,0.000000,0.003646,0.279752,0.209677,0.000000,0.000000,0.0
9,SaF5.1202D.1677983769.180831.NT1233.wav,DSa,0.004617,0.130569,0.397958,0.170428,0.000000,0.0,0.0,0.001219,...,0.0,0.060688,0.012792,0.003060,0.050715,0.284951,0.132467,0.000000,0.000970,0.0


In [8]:
all_saved_test_accs = []
ConfusionMatrix = np.zeros((num_classes,num_classes), dtype=float)

for i in range(100): #perform 10 different CV splits
  repeat = i+1 
  thisCV_saved_test_accs = []
  for g in range(1): #perform 10 repeats of each CV split
      
      train_df = pd.DataFrame(columns=df_withclasses.columns)
      val_df = pd.DataFrame(columns=df_withclasses.columns)
      test_df = pd.DataFrame(columns=df_withclasses.columns)
      
      print('Training cross val: ' + str(repeat))
      train_deployments, val_deployments, test_deployments = train_test_val_split(df)
      train_files, val_files, test_files = put_files_in_splits(train_deployments, val_deployments, test_deployments)

      for i in range(len(df_withclasses)):
        if df_withclasses['minute'][i] in train_files:
          train_df = train_df.append([df_withclasses.iloc[[i]]])


        if df_withclasses['minute'][i] in val_files:
          val_df = val_df.append([df_withclasses.iloc[[i]]])

        if df_withclasses['minute'][i] in test_files:
          test_df = test_df.append([df_withclasses.iloc[[i]]])

      train_feats = train_df.iloc[:, 2:].to_numpy()
      train_labels = train_df.iloc[:, 1].to_numpy()

      val_feats = val_df.iloc[:, 2:].to_numpy()
      val_labels = val_df.iloc[:, 1].to_numpy()

      test_feats = test_df.iloc[:, 2:].to_numpy()
      test_labels = test_df.iloc[:, 1].to_numpy()

      val_accuracy_score = 0
      for k in range(50):              # Picked 50 as 50 epochs used in NN
        model = RandomForestClassifier(n_jobs = -1, random_state=k)
        model.fit(train_feats, train_labels)
        new_val_acc = model.score(val_feats, val_labels)
        if new_val_acc >  val_accuracy_score:
          val_accuracy_score = new_val_acc
          test_acc = model.score(test_feats, test_labels)
          test_predictions = model.predict(test_feats)
          best_ConfusionMatrix = confusion_matrix(test_labels, test_predictions, labels = labels)
      thisCV_saved_test_accs.append(test_acc)
      all_saved_test_accs.append(test_acc)
      ConfusionMatrix = np.add(ConfusionMatrix, best_ConfusionMatrix)
  print('Accuracies for cross validation split number : '+ str(repeat))
  print(thisCV_saved_test_accs)
  thisCV_saved_test_accs = []
  
  
print()
print(repr(ConfusionMatrix))
  
  
print('Inddo pretrained sites completed RFs:')
print(len(all_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(all_saved_test_accs)
stdev = np.std(all_saved_test_accs)

print('saved_test_accs: ')
print(all_saved_test_accs)

result = 'Mean accuracy with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result)

Training cross val: 1
Accuracies for cross validation split number : 1
[0.8392434988179669]
Training cross val: 2
Accuracies for cross validation split number : 2
[0.8577777777777778]

array([[240.,   0.,   0.,   0.],
       [  1., 154.,  23.,   2.],
       [ 70.,  34., 135.,   1.],
       [  0.,   0.,   1., 212.]])
Inddo pretrained sites completed RFs:
2
saved_test_accs: 
[0.8392434988179669, 0.8577777777777778]
Mean accuracy with standard deviation = 0.8485106382978723 (±0.009267139479905429)


In [ ]:
#HPCconfusion_matrix = [[9.1000e+03, 2.1400e+02, 1.4390e+03, 1.2470e+03],
 #      [3.0900e+02, 1.0212e+04, 9.3100e+02, 8.0000e+00],
  #     [1.1550e+03, 8.4500e+02, 9.6390e+03, 6.4000e+01],
   #    [6.8000e+02, 1.4100e+02, 1.6100e+02, 1.0640e+04]]

array = ConfuseionMatrix
df_cm = pd.DataFrame(array, index = [i for i in "ABCD"],
                  columns = [i for i in "ABCD"])
plt.figure(figsize = (15,12))
cmap = sn.cm.rocket_r
ax = sn.heatmap(df_cm, annot=True, annot_kws={"fontsize":25}, fmt='g', cmap = cmap)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 25)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 25)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)